### User Car Price Prediction

### 1. Problem Statement
- This dataset comprises used cars sold on cardehko.com in India as well as important features of these cars.
- If user can predict the price of the car based on input features.
- Prediction results can be used to give new seller the price suggestion based on market condition.

### 2. Data Collection
- The Dataset is collected from scrapping from cardheko webiste
- The data consists of 13 column and 15411 rows.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
df = pd.read_csv('Data/cardekho_imputated.csv')
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [5]:
df.shape

(15411, 14)

##### Handling Missing Values

In [4]:
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [6]:
## Remove Unnecessary Columns
df.drop(columns=['car_name','brand'],axis=1,inplace=True)

In [7]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [8]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [9]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [10]:
X = df.drop(['selling_price'],axis=1)
y = df['selling_price']

**As model may impact the price of the Car so use `LabelEncoder()` for that**

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

In [12]:
# Create column transformer with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
onehotcolumns = ['seller_type','fuel_type','transmission_type']



from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,onehotcolumns),
        ("StandardScaler",numeric_transformer,num_features)
    ],remainder='passthrough'
)

In [13]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('OneHotEncoder', OneHotEncoder(drop='first'),
                                 ['seller_type', 'fuel_type',
                                  'transmission_type']),
                                ('StandardScaler', StandardScaler(),
                                 Index(['model', 'vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object'))])

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [15]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
# Creating function for Evaluating the Model
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2score = r2_score(true,predicted)
    return mae,mse,r2score,rmse

In [18]:
## Begenning the model

models = {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "k-Nearest Neighbour Regressor": KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest" : RandomForestRegressor(),
    "Adaboost Regressor": AdaBoostRegressor()
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    ## Make Prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae,model_train_mse,model_train_r2score,model_train_rmse = evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_mse,model_test_r2score,model_test_rmse = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])

    print("Model Performance for Training set")
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- R2 Score: {:.4f}".format(model_train_r2score))
    print("- Root-Mean Squared Error: {:.4f}".format(model_train_rmse))


    print('------------------------------------------------------------------')

    print("Model Performance for Test set")
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("-  R2 Score: {:.4f}".format(model_test_r2score))
    print("- Root-Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("\n")


Linear Regression
Model Performance for Training set
- Mean Absolute Error: 266675.1076
- Mean Squared Error: 304874315292.8461
- R2 Score: 0.6220
- Root-Mean Squared Error: 552154.2495
------------------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 284283.4460
- Mean Squared Error: 270286925822.7529
-  R2 Score: 0.6525
- Root-Mean Squared Error: 519891.2635


Lasso
Model Performance for Training set
- Mean Absolute Error: 266674.0558
- Mean Squared Error: 304874327627.3900
- R2 Score: 0.6220
- Root-Mean Squared Error: 552154.2607
------------------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 284283.8047
- Mean Squared Error: 270286283692.1716
-  R2 Score: 0.6525
- Root-Mean Squared Error: 519890.6459


Ridge
Model Performance for Training set
- Mean Absolute Error: 266635.7643
- Mean Squared Error: 304875008338.2686
- R2 Score: 0.6220
- Root-Mean Squared Error: 552154.877

##### HyperParameter Tuning

In [20]:
rf_params = {
    "max_depth":[5,18,15,None,10],
    "max_features":[5,7,"auto",8],
    "min_samples_split":[2,8,15,20],
    "n_estimators":[100,200,500,1000]
}

ada_params = {
    'n_estimators':[50,60,70,80],
    'loss' : ['linear', 'square', 'exponential']
}
knn_params = {"n_neighbors":[2,3,10,20,40,50]}

In [21]:
randomcv_models = [
    ("KNN",KNeighborsRegressor(),knn_params),
    ("RF",RandomForestRegressor(),rf_params),
    ("Adaboost",AdaBoostRegressor(),ada_params)
]

In [22]:
from sklearn.model_selection import RandomizedSearchCV

model_params = {}
for name,model,params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                param_distributions=params,
                                n_iter=100,
                                cv=3,
                                verbose=2,
                                n_jobs=1)
    
    random.fit(X_train,y_train)
    model_params[name] = random.best_params_

for model_name in model_params:
    print(f"------------------- Best Params for {model_name} --------------")
    print(model_params[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END ......................................n_neighbors=2; total time=   0.2s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=2; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.0s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END ......................................n_neighbors=3; total time=   0.1s
[CV] END .....................................n_neighbors=10; total time=   0.1s
[CV] END .....................................n_neighbors=10; total time=   0.1s
[CV] END .....................................n_neighbors=10; total time=   0.1s
[CV] END .....................................n_neighbors=20; total time=   0.2s
[CV] END .....................................n_neighbors=20; total time=   0.3s
[CV] END .....................................n_n

In [23]:
## Begenning the model

models = {
    "k-Nearest Neighbour Regressor": KNeighborsRegressor(n_neighbors=2),
    "Random Forest" : RandomForestRegressor(n_estimators=100,min_samples_split=2,max_features=7,max_depth=None),
    "Adaboost Regressor": AdaBoostRegressor()
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    ## Make Prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae,model_train_mse,model_train_r2score,model_train_rmse = evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_mse,model_test_r2score,model_test_rmse = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])

    print("Model Performance for Training set")
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- R2 Score: {:.4f}".format(model_train_r2score))
    print("- Root-Mean Squared Error: {:.4f}".format(model_train_rmse))


    print('------------------------------------------------------------------')

    print("Model Performance for Test set")
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("-  R2 Score: {:.4f}".format(model_test_r2score))
    print("- Root-Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("\n")


k-Nearest Neighbour Regressor
Model Performance for Training set
- Mean Absolute Error: 65304.0751
- Mean Squared Error: 46855041508.0464
- R2 Score: 0.9419
- Root-Mean Squared Error: 216460.2539
------------------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 120407.1827
- Mean Squared Error: 90860473985.3685
-  R2 Score: 0.8832
- Root-Mean Squared Error: 301430.7117


Random Forest
Model Performance for Training set
- Mean Absolute Error: 39592.1844
- Mean Squared Error: 23692747497.9615
- R2 Score: 0.9706
- Root-Mean Squared Error: 153924.4863
------------------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 102249.4407
- Mean Squared Error: 54108809085.5418
-  R2 Score: 0.9304
- Root-Mean Squared Error: 232613.0028


Adaboost Regressor
Model Performance for Training set
- Mean Absolute Error: 354588.0563
- Mean Squared Error: 219255325899.4665
- R2 Score: 0.7281
- Root-Mea